# Import packages and load data

In [9]:
import sys
import os
import time
sys.path.append(os.path.abspath('../..'))

import gpflow
import tensorflow as tf
from gpflow.optimizers import Scipy

from rcgp.morcgp import MOGPRegressor, MORCGPRegressor, MOGPRegressor_NC, MORCGPRegressor_NC, MORCGPRegressor_NC_fixed_weights, MORCGPRegressor_fixed_weights, MORCGPRegressor_PM
from rcgp.rcgp import RCGPRegressor
from rcgp.kernels import ConstantMean, RBFKernel, SineMean
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.covariance import MinCovDet
import pandas as pd

plt.rcParams.update({
    "text.usetex": True,         
    "font.family": "serif",       
    "text.latex.preamble": r"\usepackage{amsmath}",
    'font.size': 28,         
    'axes.labelsize': 28,    
    'axes.titlesize': 30,      # <-- Add this line for title size
    'xtick.labelsize': 24,   
    'ytick.labelsize': 24,  
    'legend.fontsize': 24,
    'lines.linewidth': 5,    
    'lines.markersize': 6   
})

c:\Users\joshu\AppData\Local\Programs\Python\Python312\Lib\site-packages\gpflow\versions.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
def generate_A(d, r=1, base_strength=1.0, noise_level=0.1, seed=None):
    if seed is not None:
        np.random.seed(seed)
    shared_component = base_strength * np.ones((d, r))
    noise = noise_level * np.random.randn(d, r)
    A = shared_component + noise
    return A

def calculate_rmse(y_true, y_pred):
    errors = y_true - y_pred
    squared_errors = errors ** 2
    mse = np.mean(squared_errors)
    rmse = np.sqrt(mse)
    return rmse

def nlpd(Y_true, mu_pred, var_pred):
    epsilon = 1e-10
    var_pred = np.maximum(var_pred, epsilon)
    
    nlpd_values = 0.5 * np.log(2 * np.pi * var_pred) + ((Y_true - mu_pred) ** 2) / (2 * var_pred)
    
    return np.mean(nlpd_values)

In [3]:
def uniform_outliers_c1(Y: np.ndarray, percent_outliers: float, start: float, end: float) -> np.ndarray:
    if not (0 <= percent_outliers <= 1):
        raise ValueError("percent_outliers must be between 0 and 1.")
    if start < 0 or end <= start:
        raise ValueError("Invalid range: ensure 0 <= start < end.")

    Y_outliers = Y.copy()
    N, D = Y.shape
    total_elements = N 
    num_outliers = int(np.round(percent_outliers * total_elements))

    row_indices = np.random.choice(N, num_outliers, replace=False)
    col_indices = np.zeros(num_outliers, dtype=int) 

    signs = np.random.choice([-1, 1], size=num_outliers)

    uniform_values = np.random.uniform(start, end, size=num_outliers) * signs

    Y_outliers[row_indices, col_indices] += uniform_values

    return Y_outliers

def asymmetric_outliers_c1(Y: np.ndarray, percent_outliers: float, start: float, end: float) -> np.ndarray:
    if not (0 <= percent_outliers <= 1):
        raise ValueError("percent_outliers must be between 0 and 1.")
    if start < 0 or end <= start:
        raise ValueError("Invalid range: ensure 0 <= start < end.")
    
    Y_outliers = Y.copy()
    N, D = Y.shape
    total_elements = N 
    num_outliers = int(np.round(percent_outliers * total_elements))

    row_indices = np.random.choice(N, num_outliers, replace=False)
    col_indices = np.zeros(num_outliers, dtype=int) 

    uniform_values = np.random.uniform(start, end, size=num_outliers)

    Y_outliers[row_indices, col_indices] += uniform_values

    return Y_outliers

def focused_outliers_c1(X, Y, percent_outliers, y_value):
    X = X.copy()
    Y = Y.copy()

    n_samples = X.shape[0]
    n_outliers = int(n_samples * percent_outliers)

    indices = np.random.choice(n_samples, size=n_outliers, replace=False)
    medians = np.median(X, axis=0)

    for idx in indices:
        Y[idx, 0] = y_value
        X[idx] = medians

    return X, Y

In [8]:
def make_X_multi(X, D=2):
    """
    X: shape (N, input_dim) - multi-dimensional input
    D: number of tasks
    """
    N, input_dim = X.shape
    X_multi = []
    
    for task in range(D):
        # Add task index as last column
        X_task = np.hstack([X, np.full((N, 1), task)])
        X_multi.append(X_task)
    
    return np.vstack(X_multi)  # Shape: (N*D, input_dim + 1)

In [ ]:
# URL of the dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx'

# Read Excel file directly from the URL
df = pd.read_excel(url)

# Extract covariates X (columns X1 to X8)
X = df.loc[:, 'X1':'X8'].to_numpy()

# Extract target variables Y (columns Y1 and Y2)
Y = df.loc[:, ['Y1', 'Y2']].to_numpy()

empty_noise = np.array([1e-6]*2)

# No outliers

In [ ]:
# Split data into train and test sets (default test size = 25%)
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.25, random_state=42
)

scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

scaler_Y = StandardScaler()
Y_train_scaled = scaler_Y.fit_transform(Y_train)
Y_test_scaled = scaler_Y.transform(Y_test)

print("X_train shape:", X_train_scaled.shape)
print("X_test shape:", X_test_scaled.shape)
print("Y_train shape:", Y_train_scaled.shape)
print("Y_test shape:", Y_test_scaled.shape)

X_train shape: (576, 8)
X_test shape: (192, 8)
Y_train shape: (576, 2)
Y_test shape: (192, 2)


In [ ]:
start_total = time.time()

# --- 1. Prepare multi-task inputs ---
X_multi_train = make_X_multi(X_train_scaled, D=2)
X_multi_test = make_X_multi(X_test_scaled, D=2)
Y_multi_train = Y_train_scaled.reshape(-1, 1, order='F')
Y_multi_test = Y_test_scaled.reshape(-1, 1, order='F')

input_dim = X_train_scaled.shape[1]  # number of features
D = 2  # number of tasks

# --- 2. Define kernel ---
base_kernel = gpflow.kernels.RBF(
    lengthscales=1.0,
    variance=1.0,
    active_dims=list(range(input_dim))
)

coregion_kernel = gpflow.kernels.Coregion(
    output_dim=D,
    rank=D,
    active_dims=[input_dim]
)

# Fix the diagonal of coregion kernel
gpflow.utilities.set_trainable(coregion_kernel.kappa, False)
coregion_kernel.kappa.assign(tf.ones_like(coregion_kernel.kappa) * 1e-6)

# Combine kernels
kernel = base_kernel * coregion_kernel

# --- 3. Build exact GP model ---
model_gpr = gpflow.models.GPR(
    data=(X_multi_train, Y_multi_train),
    kernel=kernel,
    mean_function=None
)

# Optionally, you can fix the base kernel variance as before
gpflow.utilities.set_trainable(base_kernel.variance, False)

# --- 4. Optimize hyperparameters ---
opt = Scipy()

def objective_closure_gpr():
    return -model_gpr.log_marginal_likelihood()
try:
    opt.minimize(objective_closure_gpr, model_gpr.trainable_variables, options=dict(maxiter=1000))
except Exception as e:
    print(f"Optimization failed: {e}")
    print("Try reducing maxiter or checking data shapes")

# --- 5. Predict on test data ---
mean_pred_mogp, var_pred_mogp = model_gpr.predict_y(X_multi_test)
mu_mogp, std_mogp = mean_pred_mogp.numpy().reshape(-1, D, order='F'), np.sqrt(var_pred_mogp.numpy()).reshape(-1, D, order='F')

end_total = time.time()
print(f"Total runtime: {end_total - start_total:.4f} seconds")


Total runtime: 24.3734 seconds


In [12]:
# Measure total time
start_total = time.time()

mcd = MinCovDet(support_fraction=1.0).fit(Y_train_scaled)
robust_covariance = mcd.covariance_

initial_A = generate_A(d = 2, r = 2)

morcgp = MORCGPRegressor_NC_fixed_weights(mean = 0, length_scale=1.67, noise = 0.04, A=initial_A)
predictive_mean, predictive_variances = morcgp.fit(X_train_scaled, Y_train_scaled, B_weighted=robust_covariance, noise_weighted=1e-7)

predictive_mean, predictive_variances = morcgp.optimize_loo_cv(weighted=True, print_opt_param = False, print_iter_param=False, update_weights=True)

mu_morcgp, var_morcgp = morcgp.predict(X_test_scaled)
std_morcgp = np.sqrt(var_morcgp + morcgp.noise)

end_total = time.time()
print(f"Total runtime: {end_total - start_total:.4f} seconds")

Total runtime: 36.0995 seconds


In [ ]:
# Create multi-task inputs
X_multi_train = make_X_multi(X_train_scaled, D=2)
X_multi_test = make_X_multi(X_test_scaled, D=2)
Y_multi_train = Y_train_scaled.reshape(-1, 1, order='F')
Y_multi_test = Y_test_scaled.reshape(-1, 1, order='F')

input_dim = X_train_scaled.shape[1]  # This is the key fix!
N = X_train_scaled.shape[0]
D = 2

start_total = time.time()

base_kernel = gpflow.kernels.RBF(
    lengthscales=1.0, 
    variance=1.0, 
    active_dims=list(range(input_dim)) ,
)

coregion_kernel = gpflow.kernels.Coregion(
    output_dim=D, 
    rank=D, 
    active_dims=[input_dim]  
)

gpflow.utilities.set_trainable(coregion_kernel.kappa, False)
coregion_kernel.kappa.assign(tf.ones_like(coregion_kernel.kappa) * 1e-6)

kernel = base_kernel * coregion_kernel

gpflow.utilities.set_trainable(base_kernel.variance, False)

likelihood_vgp = gpflow.likelihoods.StudentT(df=10.0)
# gpflow.utilities.set_trainable(likelihood_vgp.scale, False)
model_vgp = gpflow.models.VGP(
    data=(X_multi_train, Y_multi_train),
    kernel=kernel,
    likelihood=likelihood_vgp
)

opt = Scipy()
def objective_closure_vgp():
    return -model_vgp.maximum_log_likelihood_objective()

try:
    opt.minimize(objective_closure_vgp, model_vgp.trainable_variables, options=dict(maxiter=1000))
except Exception as e:
    print(f"Optimization failed: {e}")
    print("Try reducing maxiter or checking data shapes")

mean_pred_tmogp, var_pred_tmogp = model_vgp.predict_y(X_multi_test)
mu_tmogp, std_tmogp = mean_pred_tmogp.numpy().reshape(-1, D, order='F'), np.sqrt(var_pred_tmogp.numpy()).reshape(-1, D, order='F')
end_total = time.time()
print(f"Total runtime: {end_total - start_total:.4f} seconds")

Total runtime: 354.2806 seconds


In [19]:
rmse_mogp = calculate_rmse(Y_test_scaled, mu_mogp.reshape(-1, D, order='F'))
rmse_morcgp = calculate_rmse(Y_test_scaled, mu_morcgp)
rmse_tmogp = calculate_rmse(Y_test_scaled, mu_tmogp.reshape(-1, D, order='F'))

print("RMSE MOGP:", rmse_mogp)
print("RMSE MORCGP:", rmse_morcgp)
print("RMSE t-MOGP:", rmse_tmogp)

nlpd_mogp = nlpd(Y_test_scaled, mu_mogp.reshape(-1, D, order='F'), std_mogp.reshape(-1, D, order='F')**2)
nlpd_morcgp = nlpd(Y_test_scaled, mu_morcgp, std_morcgp**2)
nlpd_tmogp = nlpd(Y_test_scaled, mu_morcgp.reshape(-1, D, order='F'), std_tmogp.reshape(-1, D, order='F')**2)

print("NLPD MOGP:", nlpd_mogp)
print("NLPD MORCGP:", nlpd_morcgp)
print("NLPD t-MOGP:", nlpd_tmogp)

RMSE MOGP: 0.13793597718016506
RMSE MORCGP: 0.18356925293598048
RMSE t-MOGP: 0.15728571647860648
NLPD MOGP: -0.5644119045615371
NLPD MORCGP: -0.263432071578953
NLPD t-MOGP: -0.2800299288376262


# Uniform outliers

# Asymmetric outliers

In [75]:
# Split data into train and test sets (default test size = 25%)
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.25, random_state=42
)

scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

scaler_Y = StandardScaler()
Y_train_scaled = scaler_Y.fit_transform(Y_train)
Y_test_scaled = scaler_Y.transform(Y_test)

Y_train_scaled = asymmetric_outliers_c1(Y=Y_train_scaled, percent_outliers=0.1, start=6, end=12)

print("X_train shape:", X_train_scaled.shape)
print("X_test shape:", X_test_scaled.shape)
print("Y_train shape:", Y_train_scaled.shape)
print("Y_test shape:", Y_test_scaled.shape)

X_train shape: (576, 8)
X_test shape: (192, 8)
Y_train shape: (576, 2)
Y_test shape: (192, 2)


In [76]:
start_total = time.time()

# --- 1. Prepare multi-task inputs ---
X_multi_train = make_X_multi(X_train_scaled, D=2)
X_multi_test = make_X_multi(X_test_scaled, D=2)
Y_multi_train = Y_train_scaled.reshape(-1, 1, order='F')
Y_multi_test = Y_test_scaled.reshape(-1, 1, order='F')

input_dim = X_train_scaled.shape[1]  # number of features
D = 2  # number of tasks

# --- 2. Define kernel ---
base_kernel = gpflow.kernels.RBF(
    lengthscales=1.0,
    variance=1.0,
    active_dims=list(range(input_dim))
)

coregion_kernel = gpflow.kernels.Coregion(
    output_dim=D,
    rank=D,
    active_dims=[input_dim]
)

# Fix the diagonal of coregion kernel
gpflow.utilities.set_trainable(coregion_kernel.kappa, False)
coregion_kernel.kappa.assign(tf.ones_like(coregion_kernel.kappa) * 1e-6)

# Combine kernels
kernel = base_kernel * coregion_kernel

# --- 3. Build exact GP model ---
model_gpr = gpflow.models.GPR(
    data=(X_multi_train, Y_multi_train),
    kernel=kernel,
    mean_function=None
)

# Optionally, you can fix the base kernel variance as before
gpflow.utilities.set_trainable(base_kernel.variance, False)

# --- 4. Optimize hyperparameters ---
opt = Scipy()

def objective_closure_gpr():
    return -model_gpr.log_marginal_likelihood()
try:
    opt.minimize(objective_closure_gpr, model_gpr.trainable_variables, options=dict(maxiter=1000))
except Exception as e:
    print(f"Optimization failed: {e}")
    print("Try reducing maxiter or checking data shapes")

# --- 5. Predict on test data ---
mean_pred_mogp, var_pred_mogp = model_gpr.predict_y(X_multi_test)
mu_mogp, std_mogp = mean_pred_mogp.numpy().reshape(-1, D, order='F'), np.sqrt(var_pred_mogp.numpy()).reshape(-1, D, order='F')

end_total = time.time()
print(f"Total runtime: {end_total - start_total:.4f} seconds")


Total runtime: 4.7374 seconds


In [77]:
# Measure total time
start_total = time.time()

mcd = MinCovDet(support_fraction=1.0).fit(Y_train_scaled)
robust_covariance = mcd.covariance_

initial_A = generate_A(d = 2, r = 2)

morcgp = MORCGPRegressor_NC_fixed_weights(mean = 0, length_scale=1, noise = 0.05, A=initial_A)
predictive_mean, predictive_variances = morcgp.fit(X_train_scaled, Y_train_scaled, B_weighted=robust_covariance, noise_weighted=1e-7)

predictive_mean, predictive_variances = morcgp.optimize_loo_cv(weighted=True, print_opt_param = False, print_iter_param=False, update_weights=True)

mu_morcgp, var_morcgp = morcgp.predict(X_test_scaled)
std_morcgp = np.sqrt(var_morcgp + morcgp.noise)

end_total = time.time()
print(f"Total runtime: {end_total - start_total:.4f} seconds")

Total runtime: 30.5798 seconds


In [68]:
print(morcgp.length_scale, morcgp.noise)

1.0105718067017913 0.04816761305752576


In [80]:
# Create multi-task inputs
X_multi_train = make_X_multi(X_train_scaled, D=2)
X_multi_test = make_X_multi(X_test_scaled, D=2)
Y_multi_train = Y_train_scaled.reshape(-1, 1, order='F')
Y_multi_test = Y_test_scaled.reshape(-1, 1, order='F')

input_dim = X_train_scaled.shape[1]  # This is the key fix!
N = X_train_scaled.shape[0]
D = 2

start_total = time.time()

base_kernel = gpflow.kernels.RBF(
    lengthscales=1.0, 
    variance=1.0, 
    active_dims=list(range(input_dim)) ,
)

coregion_kernel = gpflow.kernels.Coregion(
    output_dim=D, 
    rank=D, 
    active_dims=[input_dim]  
)

gpflow.utilities.set_trainable(coregion_kernel.kappa, False)
coregion_kernel.kappa.assign(tf.ones_like(coregion_kernel.kappa) * 1e-6)

kernel = base_kernel * coregion_kernel

gpflow.utilities.set_trainable(base_kernel.variance, False)

likelihood_vgp = gpflow.likelihoods.StudentT(df=3.0)
# gpflow.utilities.set_trainable(likelihood_vgp.scale, False)
model_vgp = gpflow.models.VGP(
    data=(X_multi_train, Y_multi_train),
    kernel=kernel,
    likelihood=likelihood_vgp
)

opt = Scipy()
def objective_closure_vgp():
    return -model_vgp.maximum_log_likelihood_objective()

try:
    opt.minimize(objective_closure_vgp, model_vgp.trainable_variables, options=dict(maxiter=1000))
except Exception as e:
    print(f"Optimization failed: {e}")
    print("Try reducing maxiter or checking data shapes")

mean_pred_tmogp, var_pred_tmogp = model_vgp.predict_y(X_multi_test)
mu_tmogp, std_tmogp = mean_pred_tmogp.numpy().reshape(-1, D, order='F'), np.sqrt(var_pred_tmogp.numpy()).reshape(-1, D, order='F')
end_total = time.time()
print(f"Total runtime: {end_total - start_total:.4f} seconds")

Total runtime: 336.1818 seconds


In [82]:
rmse_mogp = calculate_rmse(Y_test_scaled, mu_mogp.reshape(-1, D, order='F'))
rmse_morcgp = calculate_rmse(Y_test_scaled, mu_morcgp)
rmse_tmogp = calculate_rmse(Y_test_scaled, mu_tmogp.reshape(-1, D, order='F'))

print("RMSE MOGP:", rmse_mogp)
print("RMSE MORCGP:", rmse_morcgp)
print("RMSE t-MOGP:", rmse_tmogp)

nlpd_mogp = nlpd(Y_test_scaled, mu_mogp.reshape(-1, D, order='F'), std_mogp.reshape(-1, D, order='F')**2)
nlpd_morcgp = nlpd(Y_test_scaled, mu_morcgp, std_morcgp**2)
nlpd_tmogp = nlpd(Y_test_scaled, mu_morcgp.reshape(-1, D, order='F'), std_tmogp.reshape(-1, D, order='F')**2)

print("NLPD MOGP:", nlpd_mogp)
print("NLPD MORCGP:", nlpd_morcgp)
print("NLPD t-MOGP:", nlpd_tmogp)

RMSE MOGP: 1.006416588276257
RMSE MORCGP: 0.1969785874010365
RMSE t-MOGP: 0.1718626666782293
NLPD MOGP: 1.7243622855976763
NLPD MORCGP: 0.09863780808461464
NLPD t-MOGP: -0.14731465560197426
